### To do list:
#### 1. ~~Скачать датасет *VGGSound*. Ссылка: https://huggingface.co/datasets/Loie/VGGSound/tree/main~~
#### 2. ~~Все видео-файлы заменить на случайный кадр из них~~
#### 2.1 ~~Перевести wav файлы к частоте 48кГц~~
#### 3. ~~Реализовать DataLoader wav->ipeg~~
#### 4. ~~Встроить аудио-энкодер. Ссылка: https://github.com/archinetai/archisound~~
#### 5. ~~Реализовать сценарий обучения и обучить модель~~

In [1]:
import torch
from utils.config import ModelConfig
from models.unet import UNetWithCrossAttention
from models.diffusion import Diffusion
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from utils.SoundDataset import SoundDataset
from torch.utils.data import DataLoader

In [2]:
image_path = "data/images"
sound_path = "data/sounds"

data = SoundDataset(image_path, sound_path)
train_data, val_data = torch.utils.data.random_split(data, [197889-10000, 10000])

train_loader = DataLoader(train_data, 
                          batch_size=128,
                          num_workers=8,
                          pin_memory=True,
                          shuffle=True, 
                          drop_last=True)
                         
val_loader = DataLoader(val_data, 
                        batch_size=8,
                        num_workers=8,
                        pin_memory=True,
                        shuffle=False, 
                        drop_last=True)

In [3]:
from archisound import ArchiSound

device = "cuda" if torch.cuda.is_available() else "cpu"

autoencoder = ArchiSound.from_pretrained("dmae1d-ATC64-v2").to(device)

2025-05-16 16:15:29.946067: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 16:15:30.016727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747401330.046551   11209 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747401330.056257   11209 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-16 16:15:30.125812: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
config = ModelConfig({
                    "image_size": 128,
                    "audio_ctx_dim": 32
                    })

# Инициализация
diffusion = Diffusion(timesteps=1000, image_size=128, device=device)
model = UNetWithCrossAttention(config).to(device)

train_losses = []
val_losses = []

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

unconditional_prob = 0.08

epoch = 4
for _ in tqdm(range(epoch)):
    epo_train_losses = []
    epo_val_losses = []
    model.train()
    for audio, images in train_loader:

        if torch.rand(1) < unconditional_prob:
            audio_embeds = None
        else:
            with torch.no_grad():
                audio_embeds = autoencoder.encode(audio.to(device)) # [B, d_audio, seq_len] [64, 32, 431]
                
            audio_embeds = audio_embeds.permute(0, 2, 1)  # [B, seq_len, d_audio]

        images = images.to(device)
        
        optimizer.zero_grad()
        loss = diffusion.loss_fn(model, images, audio_embeds)
        loss.backward()
        optimizer.step()

        epo_train_losses.append(loss.item())
        
    scheduler.step()

    train_losses.append(sum(epo_train_losses)/len(epo_train_losses))
    # валидация
    model.eval()
    for audio, images in val_loader:
        with torch.no_grad():
            audio_embeds = autoencoder.encode(audio.to(device))
            
            audio_embeds = audio_embeds.permute(0, 2, 1)

            # audio_embeds = torch.zeros((8, 431, 32)).to(device)
            images = images.to(device)
            
            loss = diffusion.loss_fn(model, images, audio_embeds)
    
            epo_val_losses.append(loss.item())

    val_losses.append(sum(epo_val_losses)/len(epo_val_losses))
    

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
plt.plot(train_losses, label='train')
plt.plot(val_losses, label='val')
plt.grid()
plt.legend()

In [ ]:
# plt.plot(epo_train_losses, label='train')
plt.plot(epo_train_losses, label='val')
plt.grid()
plt.legend()

In [ ]:
audio,image = next(iter(val_loader))
audio_embeds = autoencoder.encode(audio.to(device))
audio_embeds = audio_embeds.permute(0, 2, 1)

model.eval()
generated_image = diffusion.reverse_process(
    model,
    audio_embeds,
    guidance_scale=7.5,
    batch_size=8,
    use_ddim=True,
    timesteps=100  #число шагов
)

# 16 384
# 32 768

In [ ]:
plt.imshow(torch.permute(generated_image[0].cpu()*0.5+0.5, (1,2,0)).numpy())

In [ ]:
# датасет обходится за 25 мин, 40 сек
# при num_workers = 8 обходится за ~6 мин 30 сек
# 1300 сек на эпоху

# при двух attention блоках средние потери падают до 0.54 на 11 эпохах
# при единственном блоке потери не пробивали отсечку в 0.9

# 3 attention блока дали 0.84 на 4 эпохах. При продолжении обучения к концу 7 эпохи потери не изменились

In [ ]:
# for el in model.mid_block.attn.parameters():
#     print("Attention stats:", el.mean().item(), el.max().item())

In [ ]:
# torch.save(model.state_dict(), "weights2.pth")

In [ ]:
scheduler.get_lr()